#### LSTM For EEG Data
- First, import everything we're gonna be using

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
torch.manual_seed(1)
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [35]:
class EEGLSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim = 4):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim)
        self.linear = nn.Linear(self.hidden_dim, self.output_dim)
        self.hidden = self.init_hidden(self.hidden_dim)
        
        # hidden layer init
    def init_hidden(self, hidden_dim):
        """Initialize the hidden state in self.hidden
        Dimensions are num_layers * minibatch_size * hidden_dim
        """
        return (autograd.Variable(torch.zeros(1, 1, hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, hidden_dim)))
    
    def forward(self, input):
        # input is 1000 dimensional for now
        input = autograd.Variable(torch.FloatTensor(input)).contiguous()
        input = input.view(1, 1, -1) # present the sequence one at a time for now
        lstm_out, self.hidden = self.lstm(input, self.hidden)
        scores = self.linear(lstm_out.view(1,-1))
        return scores
        
        
    

In [36]:
model = EEGLSTM(22, 20)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [5]:
# load the data
from load_data import EEGDataLoader
data_loader = EEGDataLoader()
X_train, y_train, X_test, y_test = data_loader.load_all_data()

In [38]:
sample = X_train[0][0].T
label = y_train[0][0]
model.zero_grad()
model.hidden = model.init_hidden(20)
print(label)
sample[0].shape
scores = model(sample[0])
print(scores.shape)
print(scores)
label = autograd.Variable(torch.LongTensor([int(label % 769)]))
loss = loss_function(scores, label)
print(loss.data[0])

loss.backward(retain_graph = True)
optimizer.step()
scores = model(sample[1])
loss = loss_function(scores, label)
print(loss.data[0])

model.zero_grad()
model.hidden = model.init_hidden(20)
loss.backward(retain_graph = True)
optimizer.step()
torch.nn.utils.clip_grad_norm(model.parameters(), 0.25)
scores = model(sample[2])
loss = loss_function(scores, label)
print(loss.data[0])
print(torch.norm(list(model.parameters())[0].grad))

771
torch.Size([1, 4])
Variable containing:
 0.4645 -0.0190  0.0065 -0.0921
[torch.FloatTensor of size 1x4]

1.4955272674560547
1.449339747428894
1.432694911956787
Variable containing:
 0.2031
[torch.FloatTensor of size 1]



In [39]:
for epoch in range(500):
    for i in range(X_train.shape[1]):
        # get the data
        model.zero_grad()
        model.hidden = model.init_hidden(20)
        X_sample, label = X_train[0][i].T[0], y_train[0][i]
        # clear out grads and re-init hidden state
        scores = model(X_sample)
        # compute the loss, grads, and update the parameters
        target = autograd.Variable(torch.LongTensor([int(label % 769)]))
        loss = loss_function(scores, target)
        if i % 10 == 0:
            print("gradient norm: {}".format(torch.norm(list(model.parameters())[1].grad).data[0]))
            print(loss.data[0])
        loss.backward()
        optimizer.step()
        torch.nn.utils.clip_grad_norm(model.parameters(), 0.25)
    
        
        
        
        
    

gradient norm: 0.0
1.4911805391311646
gradient norm: 0.0
1.5961874723434448
gradient norm: 0.0
1.4049646854400635
gradient norm: 0.0
1.3361512422561646
gradient norm: 0.0
1.3351922035217285
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradient norm: 0.0
nan
gradi

KeyboardInterrupt: 